# Debugging DRP Outputs
This page describes several techniques to find problems and inconsistencies from DRP Processing.

The first step is to connect to the time series database.  The example below uses PostgreSQL, but this can also be done with a SQLite version.

In [1]:
from modules.quicklook.src.analyze_time_series import AnalyzeTimeSeries
from datetime import datetime
import pandas as pd
%matplotlib inline
pd.set_option('display.max_rows', None) # show all rows
pd.set_option('display.max_columns', None)  # show all columns without wrapping
pd.set_option('display.width', 200)
db_path = '/data/time_series/kpf_ts.db'
backend='psql' # one can also use backend='sqlite'
myTS = AnalyzeTimeSeries(db_path=db_path, backend=backend)

INFO: Starting AnalyzeTimeSeries
INFO: Starting KPF_TSDB
INFO: Jupyter Notebook environment detected.
INFO: Base data directory: /data/L0
INFO: Backend: psql
INFO: PSQL server: 127.0.0.1
INFO: PSQL username: timeseriesdba
INFO: PSQL user role: operations
INFO: Metadata table exists.
INFO: Metadata table read.
INFO: Data tables exist.


## Finding Observations with inconsistent DRPTAG values between 2D, L1, and L2 data
The code below returns a dataframe with the DRPTAG values for each data product.

In [2]:
start_date = datetime(2024,  4,  1)
end_date   = datetime(2024,  4, 30)
columns_to_display = ['ObsID', 'OBJECT', 'Source', 'DRPTAG2D', 'DRPTAGL1', 'DRPTAGL2', 'DATAPRL0', 'KWRDPRL0', 'TIMCHKL0', 'GOODREAD', 'DATAPR2D',]
QC_not_fail = ['DATAPRL0', 'KWRDPRL0', 'TIMCHKL0', 'GOODREAD', 'DATAPR2D']
df = myTS.db.dataframe_from_db(columns_to_display, start_date=start_date, end_date=end_date, QC_not_fail=QC_not_fail, not_junk=True)
print(f'Number of rows: {len(df)}')

Number of rows: 14377


Now let's filter this table for cases where DRPTAG values are not equal among the data levels and print the results.  The selection criterion removes observations that failed one of five Quality Control tests.  The results are printed in a table with links to Jump, a web-based portal used by the KPF Science Team.  The method `display_data()` can use another URL base with the argument `url_stub`.

In [3]:
condition_l1 = ((df['DRPTAG2D'] != df['DRPTAGL1']) & df['DRPTAG2D'].notna() & df['DRPTAGL1'].notna() )
condition_l2 = ((df['DRPTAGL1'] != df['DRPTAGL2']) & df['DRPTAGL1'].notna() & df['DRPTAGL2'].notna() )

filtered_df = df.loc[condition_l2, columns_to_display]
print(f'Number of rows: {len(filtered_df)}')
myTS.db.display_data(filtered_df)

Number of rows: 89


ObsID,OBJECT,Source,DRPTAG2D,DRPTAGL1,DRPTAGL2,DATAPRL0,KWRDPRL0,TIMCHKL0,GOODREAD,DATAPR2D
KP.20240401.09233.02,autocal-etalon-all-eve,Etalon,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09341.58,autocal-etalon-all-eve,Etalon,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09450.29,autocal-etalon-all-eve,Etalon,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09670.60,autocal-thar-sci-eve,ThAr,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09729.13,autocal-thar-sci-eve,ThAr,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09787.70,autocal-thar-sci-eve,ThAr,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09846.27,autocal-thar-sci-eve,ThAr,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.09904.85,autocal-thar-sci-eve,ThAr,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True
KP.20240401.10011.66,autocal-thar-sky-eve,ThAr,v2.10.3,v2.10.3,v2.8.2,True,True,True,True,True
KP.20240401.10070.52,autocal-thar-sky-eve,ThAr,v2.10.3,v2.10.3,v2.9.1,True,True,True,True,True


To help diagnose the problems, we can make a table of the combinations of DRPTAG by data level.

In [4]:
# Group by the three columns and count how many rows for each combination
summary = filtered_df.groupby(['DRPTAG2D', 'DRPTAGL1', 'DRPTAGL2']).size().reset_index(name='Count')

# Format and print a text table
print(f"{'DRPTAG2D':<15} {'DRPTAGL1':<15} {'DRPTAGL2':<15} {'Count':<5}")
print("-" * 55)
for _, row in summary.iterrows():
    print(f"{row['DRPTAG2D']:<15} {row['DRPTAGL1']:<15} {row['DRPTAGL2']:<15} {row['Count']:<5}")

DRPTAG2D        DRPTAGL1        DRPTAGL2        Count
-------------------------------------------------------
v2.10.3         v2.10.3         v2.8.2          3    
v2.10.3         v2.10.3         v2.9.1          86   


## Printing Errors from Log Files
The method `print_log_error_report()` searches the log files (assumed to be in `/data/logs/`, but configurable with the argument 'log_dir') and prints log lines with '[ERROR]'.  Only the first five obsevations in the dataframe are processed below.

In [5]:
myTS.db.print_log_error_report(filtered_df.head(5))

ObsID: KP.20240401.09233.02
Log file: /data/logs/20240401/KP.20240401.09233.02.log
Log modification date: 2025-06-27 17:44:49 UTC
Errors in log file:
    [KP.20240401.09233.02.log][ERROR]:Failed executing primitive AmplifierMask: too many indices for array: array is 1-dimensional, but 2 were indexed

--------------------------------------------------

ObsID: KP.20240401.09341.58
Log file: /data/logs/20240401/KP.20240401.09341.58.log
Log modification date: 2025-06-27 17:44:26 UTC
Errors in log file:
    [KP.20240401.09341.58.log][ERROR]:Failed executing primitive AmplifierMask: too many indices for array: array is 1-dimensional, but 2 were indexed

--------------------------------------------------

ObsID: KP.20240401.09450.29
Log file: /data/logs/20240401/KP.20240401.09450.29.log
Log modification date: 2025-06-27 17:44:17 UTC
Errors in log file:
    [KP.20240401.09450.29.log][ERROR]:Failed executing primitive AmplifierMask: too many indices for array: array is 1-dimensional, but 2 were

## Aggregated Error Reports
The method `print_log_error_report()` can also make an aggregated report that lists the frequency of different error messages in the log files.

In [6]:
myTS.db.print_log_error_report(filtered_df, aggregated_summary=True)


Aggregated Error Summary:



Count,Error Message
117,Problem with determining age of TRACFILE: 'AnalyzeL1' object has no attribute 'measure_master_age'
117,Problem with determining age of LAMPFILE: 'AnalyzeL1' object has no attribute 'measure_master_age'
59,"Failed executing primitive AmplifierMask: too many indices for array: array is 1-dimensional, but 2 were indexed"
29,Blaze algorithm failed: `x` must be strictly increasing sequence.
29,"Failed executing primitive RadialVelocity: Objective function has encountered a non-finite value, this will cause the fit to fail!"
1,"Failed executing primitive BaryCorrTable: operands could not be broadcast together with shapes (32,) (31,)"
